<a href="https://colab.research.google.com/github/Nganle21/Big-Data-Dreamteam/blob/main/Ngan/PF_songfeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

cid = '6c8892a3331d4602830359f009d42765'
secret = '2b147051b73d415c891fccec0d139b15'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from collections import Counter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [74]:

artist_name= 'Justin Bieber'
artistID = (sp.search(q='artist:' + artist_name, type='artist'))['artists']['items'][0]['id']
albumResults = sp.artist_albums(artistID, limit=50)
albumResults = albumResults['items']
z=0
tracks_info = []

# Loop over album
for album in albumResults:
            
    ## Extract album data and fill database
    albumID = album['id']
    album_name =  album['name']            
    #print("ALBUM: " + album['name'])

    # TRACK DETAILS
    trackResults = sp.album_tracks(albumID)
    trackResults = trackResults['items']
            
    ## Loop over tracks
    for track in trackResults:
                
        artists_names =[]
        artists_ids = []
                
        for artist in track['artists']:
            artists_names.append(artist['name'])
            artists_ids.append(artist['id'])
                
            ## Extract track data and fill database
        if (artistID in artists_ids or artist_name in artists_names):
            z+=1
            track_info = [artists_ids[0],  
                            artists_names[0], 
                            track['uri'],
                            track['name']]
            tracks_info.append(track_info)
df_tracks = pd.DataFrame(data = tracks_info,columns=['artist_id','artist_name','track_uri','track_name'])
df_tracks=df_tracks.drop_duplicates(subset=['track_name'])
df_tracks=df_tracks.drop_duplicates(subset=['track_uri'])
print(df_tracks)


                  artist_id    artist_name  \
0    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
1    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
2    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
3    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
4    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
5    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
6    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
7    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
8    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
9    1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
10   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
11   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
12   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
13   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
14   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
15   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
16   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
17   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
18   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
19   1uNFoZAHBGtllmzznpCI3s  Justin Bieber   
20   1uNFoZAHBGtllmzznpCI3s  Justi

In [75]:
audio_features = []
for uri in df_tracks['track_uri'].values:
    audio_features.append(sp.audio_features(uri)[0])
audio_features = pd.DataFrame(audio_features)[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                             'acousticness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
audio_features['duration'] = audio_features['duration_ms']/60000
audio_features = pd.concat([df_tracks['track_name'],df_tracks['track_uri'],audio_features],axis=1)
audio_features.head()

,track_name,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_ms,time_signature,duration
0,2 Much,spotify:track:6i4G85fdtDwK6UpQba337E,0.583,0.444,9.0,-8.601,1.0,0.0456,0.5930,0.3530,0.167,119.590,152796.0,4.0,2.546600
1,Deserve You,spotify:track:2t96tGXVqmNT9KbbIXyEP1,0.677,0.633,7.0,-7.831,1.0,0.0771,0.0314,0.1120,0.409,89.934,187239.0,4.0,3.120650
2,As I Am (feat. Khalid),spotify:track:5RE1BlC2bVRL02T0wcLq2X,0.595,0.543,11.0,-8.149,1.0,0.0380,0.1270,0.0975,0.109,99.928,174406.0,4.0,2.906767
3,Off My Face,spotify:track:3Ic8V1CEjOY53OVpo2k1Ap,0.509,0.228,4.0,-8.680,1.0,0.0332,0.9200,0.1040,0.586,90.674,156468.0,4.0,2.607800
4,Holy (feat. Chance The Rapper),spotify:track:2HFHwTr1IBWheGoQtNNKtx,0.670,0.693,6.0,-8.340,1.0,0.3500,0.2000,0.0909,0.388,87.017,212093.0,4.0,3.534883
